# Постановка задачи.

В данном соревновании требуется обучить модель предсказывать стоимость автомобиля по его характеристикам.

# Стартовая подготовка.

In [587]:
import os
import re
import sys
import json 
import random
import numpy as np
import pandas as pd
import seaborn as sns
import pandas_profiling

from pprint import pprint

from datetime import (
    date,
    datetime,
    timedelta,
)

from sklearn.model_selection import (
    train_test_split,
    KFold,
    GridSearchCV,
)
from tqdm.notebook import tqdm
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import (
    LogisticRegression,
    LinearRegression,
)
from sklearn.ensemble import (
    StackingRegressor,
    RandomForestRegressor,
    GradientBoostingRegressor,
)


import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option("max_column", None)

import warnings
warnings.filterwarnings("ignore")

In [588]:
# зафиксируем версию пакетов и random_seed, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

TEST_SIZE = 0.25
RANDOM_SEED = 13
random.seed(RANDOM_SEED)

# Вспомогательные функции

In [589]:
def body_type_revork(row):
    try:
        reg = re.compile(r'(l1)|(l2)|(l3)|([a-z]-)|[a-z0-146-9]')
        return reg.sub('', str(row).lower()).strip()
    except TypeError:
        return row

# Визуализация числовых данных
def veiw_numeric(column, size = 7, title=None):
    if not title:
        title = column.name
    fig, (g1, g2) = plt.subplots(1, 2, figsize = (2*size,size))
    fig.suptitle(f'Histogram and boxplot for {title} ', fontsize=14)
    g1.hist(column, bins = 20, histtype = 'bar', align = 'mid', rwidth = 0.8, color = 'red') # гистограмма
    g2.boxplot(column, vert = False)  # выбросы
    plt.figtext(0.5, 0, title, fontsize = 14, ha='center')
    plt.legend()

def fill_numeric_nan(column, min_v=None, max_v=None, replace=None): 
    if not min_v:
        min_v = column.min()
    if not max_v:
        max_v=column.max()
    if max_v < min_v:
        max_v, min_v = min_v, max_v
    
    if replace:
        if column.dtypes == 'float64':
            random_fill = column.replace(replace, round(random.uniform(min_v, max_v+0.1), 1))
        else:
            random_fill = column.replace(replace, random.randint(min_v, max_v))
        veiw_numeric(random_fill, title=f'{column.name}_random')
        mode = column.mode()[0]
        mode_fill = column.replace(replace, mode)
        veiw_numeric(mode_fill, title=f'{column.name}_mode')
    else:
        if column.dtypes == 'float64':
            random_fill = column.fillna(round(random.uniform(min_v, max_v+0.1), 1))
        else:
            random_fill = column.fillna(random.randint(min_v, max_v))
        veiw_numeric(random_fill, title=f'{column.name}_random')
        mode = column.mode()[0]
        mode_fill = column.fillna(mode)
        veiw_numeric(mode_fill, title=f'{column.name}_mode')
    return random_fill, mode_fill

def receive_engine_capacity(row):
    reg = re.compile('(\d\.\d)')
    try:
        return float(reg.search(str(row)).group())
    except AttributeError:
        return row

def receive_electro_capacity(row):
    if isinstance(row, str):
        reg = re.compile('(\d+ кВт)')
        Ah = round(float(reg.search(str(row)).group().split()[0]) * 94 / 126, 5)
        return round(Ah, 1)
    else:
        return row

def find_cat(df):
    cat_list = []
    for column in df.columns:
        if df[column].dtypes == 'object':
            cat_list.append(column)
    return cat_list

def mape(y_test, y_predicted):
    return np.mean(np.abs(y_test - y_predicted)/y_test)*100

def outliers_iqr(column):
    quartile_1, quartile_3 = np.percentile(column, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    
    new = column.loc[(column < lower_bound) | (column > upper_bound)]    
    
    print(f"{len(new)} выбросов")
    print(f'25-й перцентиль: {quartile_1},', f'75-й перцентиль: {quartile_3},', 
          f'IQR: {iqr}, ', f'Границы выбросов: [{lower_bound}, {upper_bound}].')
    
    return new

# гистограммы логарифмирования
def sqrt_log_veiw(column, a=1):
    # логарифмирование
    log = np.log(column + a)
    veiw_numeric(log, title=f"логарифм {column.name}")

    # взятие квадратного корня
    sqrt = np.sqrt(column)
    veiw_numeric(sqrt, title=f"квадратный корень {column.name}")
    
    # логарифмирование квадратного корня
    log_sqrt = np.log(sqrt + a)
    veiw_numeric(log_sqrt, title=f"логарифм квадратного корня {column.name}")
    
    # взятие квадратного корня логарифма
    sqrt_log = np.sqrt(log)
    veiw_numeric(sqrt_log, title=f"квадратный корень логарифма {column.name}")
    
    return log, sqrt, log_sqrt, sqrt_log

# Data upload

In [590]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [591]:
VERSION    = 1
DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.25   # 25%


In [592]:
df_train = pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
df_test = pd.read_csv(DIR_TEST+'test.csv') # датасет для тестирования модели
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

# Raw Data Inspection. Осмотр сырых данных.

## train

### размер

In [593]:
print('train', df_train.shape)
print('test', df_test.shape)

### описание

In [594]:
display('train')
display(df_train.info())
print('\n')
display('test')
display(df_test.info())

### статистики

In [595]:
display('train', df_train.describe().round(2))
display('test', df_test.describe().round(2))

### уникальные значения

In [596]:
display('train')
display(df_train.nunique())
print('\n')
display('test')
display(df_test.nunique())

### пример

In [597]:
display('train', df_train.sample(5))
print('\n')
display('test', df_test.sample(5))

Для точности, приведём названия признаков к единому формату.

In [598]:
df_train.rename(columns={'bodyType': 'body_type',
                         'fuelType': 'fuel_type',
                         'modelDate': 'model_date',
                         'name':'modification',
                         'numberOfDoors': 'number_of_doors',
                         'productionDate': 'prod_date',
                         'vehicleConfiguration': 'vehicle_config',
                         'vehicleTransmission': 'vehicle_transmission',
                         'engineDisplacement': 'engine_capacity',
                         'enginePower': 'engine_power',
                         'Комплектация': 'specification',
                         'Привод': 'type_of_drive',
                         'Руль': 'steering_wheel',
                         'Состояние': 'condition',
                         'Владельцы': 'owners',
                         'ПТС': 'licence',
                         'Таможня': 'customs',
                         'Владение': 'ownership_time',
                         'Price': 'price',
                        }, inplace=True)

In [599]:
df_test.rename(columns={'bodyType': 'body_type',
                        'complectation_dict': 'specification',
                        'engineDisplacement': 'engine_capacity',
                        'enginePower': 'engine_power',
                        'equipment_dict': 'equipment',
                        'fuelType': 'fuel_type',
                        'modelDate': 'model_date',
                        'name':'modification',
                        'numberOfDoors': 'number_of_doors',
                        'priceCurrency': 'price_currency',
                        'productionDate': 'prod_date',
                        'vehicleConfiguration': 'vehicle_config',
                        'vehicleTransmission': 'vehicle_transmission',
                        'Владельцы': 'owners',
                        'Владение': 'ownership_time',
                        'ПТС': 'licence',
                        'Привод': 'type_of_drive',
                        'Руль': 'steering_wheel', 
                        'Состояние': 'condition', 
                        'Таможня': 'customs',
                        }, inplace=True)

## пример submission

In [600]:
display(sample_submission.sample(5))

Определим какие признаки есть в обоих наборах данных одновременно

In [601]:
cross_features = list(set(df_test.columns).intersection(df_train.columns))
print('Совпадающие признаки в обоих датасетах:\n', ',\n'.join(cross_features))

Не имеет смысла обучаться на данных которых нет в test. Аналогично не имеют значимости признаки, которых нет в обучающем датасете. Поэтому, оставим в датасетах только те признаки, которые есть в обоих, но после обработки данных в train добавим price, а в test добавим cell_id для submission.

In [602]:
train = df_train[cross_features]
test = df_test[cross_features]

Объединим датасеты

In [603]:
train['sample'] = 0
test['sample'] = 1
train['price'] = df_train['price']
test['price'] = 0

df = train.append(test, sort=False).reset_index(drop=True)

In [604]:
df.sample(5)

# EDA. Разведывательный анализ.

In [605]:
pandas_profiling.ProfileReport(df.drop("sample", axis=1))

 - mileage - пробег автомобиля. Числовой признак, есть выбросы, пропусков нет.
 - modification - содержит информацию о других признаках: объём, мощность двигателя. Пропусков мало.
 - description - комментарий. Пропусков много.
 - steering_wheel - расположение руля. Бинарный, пропусков нет, одинаковые данные записаны по-разному.
 - vehicle_config - содержит информацию о других признаках: тип кузова, трансмиссия, объём двигателя. Пропусков мало.
 - ownership_time - время владения автомобилем. Очень много пропусков.
 - model_date - год выпуска авто. Числовой признак, пропусков мало.
 - type_of_drive - привод автомобиля. Пропусков мало.
 - condition - комментарий о состоянии автомобиля. Очень много пропусков.
 - engine_power - мощность двигателя. Пропусков мало.
 - licence - ПТС. Бинарный признак, пропусков мало, одинаковые данные записаны по-разному.
 - customs - растаможенн ли автомобиль.
 - number_of_doors - количество дверей. Числовой признак, пропусков мало, одинаковые данные записаны по-разному.
 - body_type - тип кузова. Пропусков мало, одинаковые данные записаны по-разному.
 - color - цвет автомобиля. Пропусков нет, одинаковые данные записаны по-разному.
 - vehicle_transmission - трансмиссия. Пропусков мало, одинаковые данные записаны по-разному.
 - owners - количество владельцев. Числовой признак, пропусков много.
 - specification - словарь, содержащий перечень комплектации автомобиля. Пропусков очень много.
 - fuel_type - тип топлива. Пропусков нет.
 - prod_date - дата последней продажи. Числовой признак, пропусков нет.
 - engine_capacity - объём двигателя. Числовой признак, пропусков мало, одинаковые данные записаны по-разному.
 - brand - марка автомобиля. Пропусков нет.
 - price - наш таргет.

# Primary visualization. Первичная визуализация.

## Осмотрим пропуски.

In [606]:
df.isnull().sum()

In [607]:
fig, ax = plt.subplots(figsize=(17,13))
sns_heatmap = sns.heatmap(df.isnull(), yticklabels=False, cbar=False)

В четырёх признаках крайне много пропусков - specification, condition, owners, ownership_time, description.

# Data porcessing. Обработка данных.

Осмотр показал что многие данные имеют разный формат при одних и тех же значениях. Приведём каждый признак к единому формату.

In [608]:
df['vehicle_transmission'].unique()

In [609]:
replace_dict = {
    'MECHANICAL': 'mechanical', 
    'AUTOMATIC': 'automatic', 
    'ROBOT': 'robot', 
    'VARIATOR': 'variator',
    'роботизированная': 'robot', 
    'механическая': 'mechanical', 
    'автоматическая': 'automatic', 
    'вариатор': 'variator'
}
df['vehicle_transmission'] = df['vehicle_transmission'].apply(lambda i: replace_dict[i] if i is not np.nan else i)

In [610]:
print(str(df['licence'].unique()))

In [611]:
replace_dict = {
    'ORIGINAL': 1, 
    'DUPLICATE': 0, 
    'Оригинал': 1, 
    'Дубликат': 0
}
df['licence'] = df['licence'].apply(lambda i: replace_dict[i] if i is not np.nan else i)

In [612]:
print(str(df['type_of_drive'].unique()))

In [613]:
replace_dict = {
    'полный': 'four-wheel', 
    'передний': 'front-wheel', 
    'задний': 'rear', 
}
df['type_of_drive'] = df['type_of_drive'].apply(lambda i: replace_dict[i] if i is not np.nan else i)

In [614]:
print(str(df['engine_power'].unique()))

In [615]:
df['engine_power'] = df['engine_power'].apply(lambda i: float(str(i).split()[0]) if i is not np.nan and not isinstance(i, float) else i)

In [616]:
print(str(df['owners'].unique()))

In [617]:
df['owners'] = df['owners'].apply(lambda i: float(str(i).split()[0]) if i is not np.nan and not isinstance(i, float) else i)

In [618]:
print(str(df['brand'].unique()))

In [619]:
df['brand'] = df['brand'].apply(lambda i: str(i).lower())

In [620]:
print(str(df['steering_wheel'].unique()))

In [621]:
replace_dict = {
    'LEFT': 1, 
    'RIGHT': 0, 
    'Левый': 1, 
    'Правый': 0
}
df['steering_wheel'] = df['steering_wheel'].apply(lambda i: replace_dict[i] if i is not np.nan else i)

In [622]:
print(str(df['color'].unique()))

In [623]:
replace_dict = {
    '040001': 'black',
    'EE1D19': 'red',
    '0000CC': 'dark_blue',
    'CACECB': 'silver',
    '007F00': 'green',
    'FAFBFB': 'white',
    '97948F': 'gray',
    '22A0F8': 'blue',
    '660099': 'purpule',
    '200204': 'brown',
    'C49648': 'beige',
    'DEA522': 'golden',
    '4A2197': 'violet',
    'FFD600': 'yelow',
    'FF8649': 'orange',
    'FFC0CB': 'pink',
    'синий': 'dark_blue',
    'чёрный': 'black',
    'серый': 'gray',
    'коричневый': 'brown',
    'белый': 'white',
    'пурпурный': 'purpule',
    'бежевый': 'beige',
    'серебристый': 'silver',
    'красный': 'red',
    'зелёный': 'green',
    'жёлтый': 'yelow',
    'голубой': 'blue',
    'оранжевый': 'orange',
    'фиолетовый': 'violet',
    'золотистый': 'golden',
    'розовый': 'pink',
}
df['color'] = df['color'].apply(lambda i: replace_dict[i] if i is not np.nan else i)

In [624]:
print(str(df['body_type'].unique()))

In [625]:

df['body_type'] = df['body_type'].apply(lambda row: body_type_revork(row))


In [626]:
print(str(df['engine_capacity'].unique()))

Признак содержит много лишней информации. Попробуем восстановить данные из признаков.

In [627]:
df['customs'].unique()

Не смотря на различие представления, признак содержит во всех записях одно и то же значение. Признак можно удалить.

In [628]:
df = df.drop('customs', axis=1)

Обработаем признаки с пропусками:
- пройдёмся по каждому признаку, содержащему пропуски;
- оценим процентное соотношение с общим числом записей; 
- выберем подходящий способ избавиться от пропусков.

In [629]:
for i in df:
    null_count = df[i].isnull().sum()
    if null_count > 0:
        print(f"{i}: {round((null_count/df.shape[0])*100, 4)}% пропущенных значений.")

Рассмотрим каждый признак из этого списка в отдельности.  
Начнём с начала.

description - признак содержит комментарий к лоту.
Пропусков довольно много, поэтому признак стоит удалить. Однако, прежде, можно выделить из него несколько признаков, например, количество слов в комментарии, общая длина комментария, наличие комментария.

Количество слов в комментарии.

In [630]:
df['count_word_description'] = df['description'].apply(lambda i: len(str(i).split()))
df['count_word_description'].sample(5)

Общая длина комментария.

In [631]:
df['len_description'] = df['description'].apply(lambda i: len(str(i)))
df['len_description'].sample(5)

Наличие комментария.

In [632]:
df['description_comment'] = df['description'].apply(lambda i: int(i is not np.nan))
df['description_comment'].unique()

In [633]:
df = df.drop('description', axis=1)

In [634]:
df['model_date'].unique()

Посмотрим на распределение признака пи заполнении модой и случайными значениями в диапазоне от min до max.

In [635]:
random_fill, mode_fill = fill_numeric_nan(df['model_date'])

Визуально распределения не различимы. Выберем вариант с заполнением случайными величинами.

In [636]:
df['model_date'] = random_fill

In [637]:
df['condition'].unique()

condition - признак описывает состояние авто.
Пропусков чрезвычайно много. Признак однозначно подлежит удалению, но, прежде, выделим признак condition comment, который характеризует наличие или отсутствие комментария condition.

In [638]:
df['condition_comment'] = df['condition'].apply(lambda i: int(isinstance(i, str)))
df['condition_comment'].unique()

In [639]:
df = df.drop('condition', axis=1)

In [640]:
print(df['modification'].unique())

modification - комбинированный признак, содержащий несколько параметров атомобиля один из которых - объём двигателя, подходит для восстановления признака engine_capacity. В остальном признак не несёт полезной информации и, не смотря на малое количество пропусков, признак подлежит удалению.

In [641]:
df['type_of_drive'].unique()

type_of_drive - заполним наиболее частым значением.

In [642]:
mode = df['type_of_drive'].mode()
df['type_of_drive'].fillna(mode[0], inplace=True)

In [643]:
df['licence'].unique()

licence - заполним модой

In [644]:
mode = df['licence'].mode()
df['licence'].fillna(mode[0], inplace=True)
df['licence'] = df['licence'].astype(int)

In [645]:
df['engine_power'].unique()

engine_power - признак означает мощность двигателя в лошадиных силах.  
Посмотрим распределение заполнений случайными велиинами (от min до max) и модой.

In [646]:
random_fill, mode_fill = fill_numeric_nan(df['engine_power'])

Пропусков на столько мало, что различные варианты заполнений ни как не влияют на распределение признака. Выберем вариант с заполнением случайными величинами.

In [647]:
df['engine_power'] = random_fill

In [648]:
df['owners'].unique()

owners - количество владельцев. Пропусков больше 10%, однако удалять признак не стоит, и удалять записи с пропуском в данном признаке тоже. Можно заполнить пропуски случайными значениями в диапазоне от 1 до 3 (целыми значениями) или модой. Рассмотрим оба варианта и выберем тот, где распределение будет равномернее.

In [649]:
df['owners'].fillna(5, inplace=True)
df['owners'] = df['owners'].apply(lambda i: int(i))
random_fill, mode_fill = fill_numeric_nan(df['owners'], replace=5)

Очевидно что распределение лучше в случае с заполнением рандомными значениями.

In [650]:
df['owners'] = random_fill

In [651]:
df['vehicle_transmission'].unique()

vehicle_transmission - тип трансмиссии авто. Заполним модой.

In [652]:
mode = df['vehicle_transmission'].mode()
df['vehicle_transmission'].fillna(mode[0], inplace=True)

In [653]:
df['number_of_doors'].unique()

Поступим уже привычным способом - посмотрим распределение заполнения модой и случайными величинами.

In [654]:
random_fill, mode_fill = fill_numeric_nan(df['number_of_doors'])

Выберем заполнение случайными величинами.

In [655]:
df['number_of_doors'] = random_fill

In [656]:
df['engine_capacity'].unique()

In [657]:
df['engine_capacity'] = df['modification'].apply(receive_engine_capacity)
print(df['engine_capacity'].unique())

В данных пристутствуют электромобили, для которых не указана ёмкость батареи (можно считать аналогом объёма двигателя). В имеющихся данных есть два примера ёмкости батареи:  
- 94Ah Electro AT (126 кВт);
- 30kWh Electro AT (81 кВт).

В киловаттчасах выражается полная ёмкость аккумуляторной батареи авто, а вот в амперчасах выражается ёмкость из расчёта величины тока, который может обеспечить аккумуляторная батарея за 1 час.
Возьмём этот показатель (амперчас) за основу и с помощью пропорции рассчитаем примерные значения ёкости.

In [658]:
df['engine_capacity'] = df['engine_capacity'].apply(receive_electro_capacity)

In [659]:
random_fill, mode_fill = fill_numeric_nan(df['engine_capacity'].apply(receive_electro_capacity))

In [660]:
df['engine_capacity'] = random_fill
df = df.drop('modification', axis=1)

ownership_time - признак содердит слишком много пропусков и воостановить данные из других признаков не представляется возможным. Удалим его.

In [661]:
df = df.drop('ownership_time', axis=1)

specification - так же как и предыдущий признак содержит много пропусков. Более того, содержит даные, которые есть в других признаках. Удалим его.

In [662]:
df = df.drop('specification', axis=1)

In [663]:
df['vehicle_config'].unique()

vehicle_config - комбинированный признак, содержит данные, представленные другими признаками. Подлежит удалению.

In [664]:
df = df.drop('vehicle_config', axis=1)

Наш таргет имеет немного пропусков. Заполним их.

In [665]:
random_fill, mode_fill = fill_numeric_nan(df['price'].loc[df['sample'] == 0])

In [666]:
df['price'] = random_fill
df['price'].fillna(0, inplace=True)

# Feature design. Проектирование признаков.

## Восстановим признак vendor по имеющимся данным и информации из интернета. 

In [667]:
df_test['vendor'].isnull().sum()

Пропусков нет.

In [668]:
group = list(pd.DataFrame(df_test.groupby(['vendor', 'brand']))[0])
vendor_dict = {i:j for j,i in group}
pprint(vendor_dict)

train['brand'].unique()

In [669]:
vendor_dict = {'AUDI': 'EUROPEAN', 'BMW': 'EUROPEAN','CADILLAC': 'USA',
            'CHERY': 'CHINESE', 'CHEVROLET': 'USA','CHRYSLER': 'USA',
            'CITROEN': 'EUROPEAN', 'DAEWOO': 'KOREAN', 'DODGE': 'USA',
            'FORD': 'USA', 'GEELY': 'CHINESE', 'HONDA': 'JAPANESE',
            'HYUNDAI': 'KOREAN', 'INFINITI': 'JAPANESE', 'JAGUAR': 'EUROPEAN',
            'JEEP': 'USA', 'KIA': 'KOREAN', 'LEXUS': 'JAPANESE',
            'MAZDA': 'JAPANESE', 'MINI': 'EUROPEAN', 'MITSUBISHI': 'JAPANESE',
            'NISSAN': 'JAPANESE', 'OPEL': 'EUROPEAN', 'PEUGEOT': 'EUROPEAN',
            'PORSCHE': 'EUROPEAN', 'RENAULT': 'EUROPEAN', 'SKODA': 'EUROPEAN',
            'SUBARU': 'JAPANESE', 'SUZUKI': 'JAPANESE', 'TOYOTA': 'JAPANESE',
            'VOLKSWAGEN': 'EUROPEAN', 'VOLVO': 'EUROPEAN', 'GREAT_WALL': 'CHINESE',
            'LAND_ROVER': 'EUROPEAN', 'MERCEDES': 'EUROPEAN', 'SSANG_YONG': 'KOREAN'}
train['vendor'] = train['brand'].map(vendor_dict)

Объединим серии и внесём признак в общий датасет.

In [670]:
vendor = train['vendor'].append(df_test['vendor']).reset_index(drop=True)
df['vendor'] = vendor

Заколируем категориальные признаки.

In [671]:
cat_columns = find_cat(df)

df = pd.get_dummies(df, columns=cat_columns)

# for colum in cat_columns:
#     df[colum] = df[colum].astype('category').cat.codes

# Корреляционный анализ.

In [672]:
# выделим все числовые признаки в список.
numeric_list = []
for col in df.columns:
    if df[col].dtypes in ['int64', 'float64'] and len(df[col].unique()) > 2:
        numeric_list.append(col)

In [673]:
sns.set(rc = {'figure.figsize':(19,11)})
sns.heatmap(df[numeric_list].corr(), 
            annot = True, fmt='.1g', linewidths = 1, cmap='coolwarm')

Многие признаки коррелируют между собой. Исправим это.

Признаки len_description и count_word_description полностью скоррелированы между собой.  
Оставим count_word_description.

In [674]:
df = df.drop('len_description', axis=1)

prod_date и model_date также полностью скоррелированы.
Оставим model_date.

In [675]:
df = df.drop('prod_date', axis=1)

Пройдёмся по числовым признакам и уберём выбросы.

In [676]:
numeric_list = []
for col in df.columns:
    if df[col].dtypes in ['int64', 'float64'] and len(df[col].unique()) > 2:
        numeric_list.append(col)
# посмотрим гистограммы
for col in numeric_list:
    veiw_numeric(df[col])

Уберём выбросы в milleage.

In [677]:
outliers = outliers_iqr(df['mileage'])

In [678]:
veiw_numeric(df.drop(outliers.index)['mileage'])

In [679]:
sqrt_log = sqrt_log_veiw(df['mileage'])

Дабы не терять данные возьмём квадратный корень признака.

In [680]:
df['mileage'] = sqrt_log[1]

Попробуем уменьшить масштаб engine_capacity.

In [681]:
# гистограммы логарифмирования
sqrt_log = sqrt_log_veiw(df['engine_capacity'], a=1)

Лучший результат показал квадратный корень логарифма.

In [682]:
df['engine_capacity'] = sqrt_log[3]

In [683]:
sqrt_log = sqrt_log_veiw(df['price'])

In [684]:
df['price'] = sqrt_log[0]

Повторно посмотрим корреляцию.

In [685]:
sns.set(rc = {'figure.figsize':(19,11)})
sns.heatmap(df[numeric_list].corr(), 
            annot = True, fmt='.1g', linewidths = 1, cmap='coolwarm')

In [686]:
df = df.drop(['engine_capacity', 'model_date', 'count_word_description'], axis=1)

### Разделим датасеты.

In [687]:
train = df.loc[df["sample"] == 0].drop("sample", axis=1)
test = df.loc[df["sample"] == 1].drop(['sample', 'price'], axis=1)

# разделим данные
X = train.drop(['price'], axis = 1)
y = train['price'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, 
                                                    shuffle=True, random_state=RANDOM_SEED)

# Naive model. Наивная модель.

Наивная модель будет предсказывать цену авто по медианному пробегу.

In [688]:
predict = X_test['mileage'].map(train.groupby('mileage')['price'].median())
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict)):0.2f}%")

Даже такая примитивная модель поазывает отличные результаты.

Попробуем побить этот результат.

# Research. Исследование.

Следующие 4 модели были выбраны как самые понятные, в надежде что они покажут хороший результат. Больше всего надежд возлагается на CatBoost.

## Простая линейная регрессия

In [689]:
linear_regr = LinearRegression().fit(X_train, y_train)
lin_predict = linear_regr.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, lin_predict)):0.2f}%")

Линейная регрессия прекрасно справляется с поставленной задачей.

## Cat_Boost

In [690]:
cat_boost_model = CatBoostRegressor(iterations = 700,
                       random_seed = RANDOM_SEED,
                       eval_metric='MAPE',
                       custom_metric=['R2', 'MAE'],
                       silent=True,
                       learning_rate=0.13, depth=12,
                       l2_leaf_reg=8, random_strength=0.3)

cat_boost_model.fit(X_train, y_train,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         plot=True
         )

# оцениваем точность
predict_cb = cat_boost_model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_cb)):0.2f}%")

Как и ожидалось CatBoost показывает отличный результат.

Предполагаю что данная модель будет лучшей.

## Случайный лес.

In [691]:
rfr_model = RandomForestRegressor(random_state=RANDOM_SEED,
                                  n_estimators=300,
                                  min_samples_split=2,
                                  min_samples_leaf=1,
                                  max_features='sqrt',
                                  max_depth=None,
                                  bootstrap=False)

rfr_model.fit(X_train, y_train)


predict_rfr = rfr_model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_rfr)):0.2f}%")

Случайный лес дал хороший результат, за малым хуже CatBoost.

## Градиентный бустинг

In [692]:
gbr_model = GradientBoostingRegressor(random_state=RANDOM_SEED,
                                      n_estimators=800,
                                      min_samples_split=5,
                                      min_samples_leaf=4,
                                      max_features='sqrt',
                                      max_depth=9)
gbr_model.fit(X_train, y_train)

predict_gbr = gbr_model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_gbr)):0.2f}%")

Градиентный бустинг так же превосходно справился со своей задачей, но всё же несколько хуже CatBoost.

# Final model. Финальная модель.

Все модели показали отличный результат.  
В качестве финальной модели выберем CatBoost потому что она показала наилучший результат.

In [693]:
subm =np.exp(cat_boost_model.predict(test)).astype('int')
subm

In [694]:
sample_submission['price'] = subm
sample_submission.to_csv('submission.csv', index=False)

# Выводы
Были изучены новые алгоритмы и обучена модель, которая показывает результат высокого уровня.  
Можно считать что поставленная задача выполнена.